In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

In [2]:
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1.0/255)

train_data = train_datagen.flow_from_directory('train_df',
                                              target_size=(128, 128),
                                              batch_size=32,
                                              class_mode='binary')

test_data = test_datagen.flow_from_directory('test_df',
                                            target_size=(128, 128),
                                            batch_size=32,
                                            class_mode='binary')

Found 1125 images belonging to 2 classes.
Found 65 images belonging to 2 classes.


In [3]:
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(128, 128, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))

c:\Users\gsuma\OneDrive\Documents\New folder\dp\env\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [4]:
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [5]:
import PIL
print(PIL.__version__)

10.4.0


In [7]:
from PIL import Image
from keras.preprocessing.image import load_img

%pip install scipy
model.fit(train_data,
          steps_per_epoch=train_data.samples // 32,
          epochs=3,
          validation_data=test_data,
          validation_steps=test_data.samples // 32)

     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     -------------------------- ------------- 41.0/60.8 kB 1.9 MB/s eta 0:00:01
     -------------------------------------- 60.8/60.8 kB 802.0 kB/s eta 0:00:00
   ---------------------------------------- 0.0/44.7 MB ? eta -:--:--
   ---------------------------------------- 0.1/44.7 MB 5.1 MB/s eta 0:00:09
   ---------------------------------------- 0.3/44.7 MB 3.5 MB/s eta 0:00:13
   ---------------------------------------- 0.4/44.7 MB 2.9 MB/s eta 0:00:16
    --------------------------------------- 0.6/44.7 MB 3.6 MB/s eta 0:00:13
    --------------------------------------- 1.1/44.7 MB 5.0 MB/s eta 0:00:09
   - -------------------------------------- 1.2/44.7 MB 4.4 MB/s eta 0:00:10
   - -------------------------------------- 1.6/44.7 MB 5.1 MB/s eta 0:00:09
   - -------------------------------------- 2.2/44.7 MB 6.0 MB/s eta 0:00:08
   -- ------------------------------------- 2.6/44.7 MB 6.4 MB/s eta 0:00:07
   --

c:\Users\gsuma\OneDrive\Documents\New folder\dp\env\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


35/35 ━━━━━━━━━━━━━━━━━━━━ 102s 2s/step - accuracy: 0.5060 - loss: 0.6938 - val_accuracy: 0.3906 - val_loss: 0.7072
Epoch 2/3
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5625 - loss: 0.7035 - val_accuracy: 1.0000 - val_loss: 0.6534
Epoch 3/3


C:\Program Files\Python311\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


35/35 ━━━━━━━━━━━━━━━━━━━━ 77s 2s/step - accuracy: 0.5819 - loss: 0.6721 - val_accuracy: 0.5000 - val_loss: 0.7144


In [8]:
loss, accuracy = model.evaluate(test_data)
print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy:.4f}')

3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 370ms/step - accuracy: 0.4805 - loss: 0.7185
Test loss: 0.7152
Test accuracy: 0.4923


In [26]:
model.save('model.h5')

In [25]:
#classifying the image
from tensorflow.keras.models import load_model
model = load_model('DR.h5')
def preprocess_image(image_path):
    from tensorflow.keras.preprocessing import image
    img = image.load_img(image_path, target_size=(128, 128))
    img = image.img_to_array(img)
    img = img / 255.0 
    img = np.expand_dims(img, axis=0) 
    return img
def predict_image(image_path):
    img = preprocess_image(image_path)
    prediction = model.predict(img)

    if prediction > 0.5:
        return 'RIPPEN'
    else:
        return 'RAW'

image_path = r'test_df\ripen\thermal\21.jpg'
result = predict_image(image_path)
print(f'The image is a {result}')
image_path


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
The image is a RIPPEN


'test_df\\ripen\\thermal\\21.jpg'

In [23]:
import os
from sklearn.metrics import *

test_dir = r'test_df'
true_labels = []
predicted_labels = []

for class_name in ['RIPPEN', 'RAW']:
    class_dir = os.path.join(test_dir, class_name)
    if os.path.exists(class_dir):
        for image_filename in os.listdir(class_dir):
            image_path = os.path.join(class_dir, image_filename)
            
            # Check if the image_path is a file
            if os.path.isfile(image_path):
                true_labels.append(class_name)
                result = predict_image(image_path)
                predicted_labels.append(result)

if true_labels and predicted_labels:
    class_report = classification_report(true_labels, predicted_labels)
else:
    class_report = "No data available for classification report."
